In [10]:
import os

def GetMoves(sgf):
    tmp=''
    for i,j in enumerate(sgf):
        if j=='(':
            tmp+=GetMoves(sgf[i:])
        elif j==')':
            tmp+=j
            return tmp
        else:
            tmp+=j
    return tmp

def ReadGame(game,path='./GoDat/KGS/'):
    g=open(path+game)
    return g.read()

def StripComments(game,path='/Users/nborggren/Codes/GamePDEs/Go/GoDat/KGS/'):
    os.system('perl CommentFilter.pl '+path+game+' '+path+game.replace('.sgf','_nocom.sgf'))
    
def GetKey(sgf,key='PW'):
    a=sgf.find(key+'[')
    b=sgf[a:].find(']')
    return sgf[a+len(key)+1:a+b]  

def StripKey(sgf,key='CR'):
    while sgf.find(key+'[')>0:
        tmp=GetKey(sgf,key=key)
        sgf=sgf.replace(key+'['+tmp+']','')
    return sgf

def CleanSGF(game,path='/Users/nborggren/Codes/GamePDEs/Go/GoDat/KGS/',write=1):
    #StripComments(game,path=path)
    #sgf=ReadGame(game.replace('.sgf','_nocom.sgf'),path=path)
    sgf=ReadGame(game,path=path)
    sgf=sgf.replace('\n','')
    for i in ['CR','WL','BL','OB','OW','C','RU','KM','TM','OT','PW','PB','WR','BR','DT','PRE','RE']:
        sgf=StripKey(sgf,key=i)
    if write==1:
        g=open(path+game.replace('.sgf','_clean.sgf'),'w')
        g.write(sgf)
    return sgf

def GetCorners(sgf):
    c1=[]
    c2=[]
    c3=[]
    c4=[]
    moves=sgf.split(';')
    moves=moves[2:]
    locs=[i[2:4] for i in moves]
    letters='abcdefghijklmnopqrs'
    bletters='srqponmlkjihgfedcba'
    sym={}
    for i,j in zip(letters,bletters):
        sym[i]=j
    l,r=letters[0:10],letters[9:]
    for i,j in zip(moves,locs):
        if j[0] in l and j[1] in l:
            c1.append(i)
        if j[0] in l and j[1] in r:
            tmp=i.replace(j[1],sym[j[1]])
            c2.append(tmp)
        if j[0] in r and j[1] in r:
            tmp=i.replace(j[1],sym[j[1]])
            tmp=tmp.replace(j[0],sym[j[0]])
            c3.append(tmp)
        if j[0] in r and j[1] in l:
            tmp=i.replace(j[0],sym[j[0]])
            c4.append(tmp)
    return [c1,c2,c3,c4]

def WriteCorners(corner,path='./GoDat/Corners/',name='test'):
    if len(corner)==0:return
    header="(;GM[1]FF[4]CA[UTF-8]ST[2]\nSZ[19]\n;"
    record=header
    for i in corner[:-1]:record+=i+';'
    record+=corner[-1].replace(')','')+')'
    f=open(path+name+'.sgf','w')
    f.write(record)
    f.close()

def GetSyms(sgf,src='notlist'):
    c1=[]
    c2=[]
    c3=[]
    c4=[]
    c5=[]
    c6=[]
    c7=[]
    c8=[]
    if src=='notlist':
        moves=sgf.split(';')
        moves=moves[2:]
    else:
        moves=sgf
    #print moves
    locs=[i[2:4] for i in moves]
    #print locs
    letters='abcdefghijklmnopqrs'
    bletters='srqponmlkjihgfedcba'
    sym={}
    for i,j in zip(letters,bletters):
        sym[i]=j
    for i,j in zip(moves,locs):
        c1.append(i)
        tmp=j[0]+sym[j[1]]
        c2.append(i.replace(j,tmp))
        tmp=sym[j[0]]+sym[j[1]]
        c3.append(i.replace(j,tmp))
        tmp=sym[j[0]]+j[1]
        c4.append(i.replace(j,tmp))
        tmp=j[1]+j[0]
        c5.append(i.replace(j,tmp))
        tmp=j[1]+sym[j[0]]
        c6.append(i.replace(j,tmp))
        tmp=sym[j[1]]+sym[j[0]]
        c7.append(i.replace(j,tmp))
        tmp=sym[j[1]]+j[0]
        c8.append(i.replace(j,tmp))
    return [c1,c2,c3,c4,c5,c6,c7,c8]

def PatternSearch(pattern,games,path='./GoDat/KGS/'):
    m=len(pattern)
    patterns=[]
    mygames=[]
    rev = {}
    rev['B']='W'
    rev['W']='B'
    for i in games:
        try:
            sgf = ReadGame(i.replace('.sgf','_clean.sgf'),path=path)
            corners=GetCorners(sgf)
        except IndexError:
            continue
        for j in corners:
            a=GetSyms(j,src='list')
            for k in a:
                if k[0:m]==pattern or k[0:m]==[rev[l[0]]+l[1:] for l in pattern]:
                    mygames.append(i)
                    patterns.append(k)
    return patterns,mygames


def PatternSearchCorners(pattern,corners,games,path='/Users/nborggren/Codes/ipy/go/go4go/'):
    games=[]
    patterns=[]
    rev = {}
    rev['B']='W'
    rev['W']='B'
    for i,mycorners in zip(games,corners):
        for j in mycorners:
            a=GetSyms(j,src='list')
            print a
            for k in a:
                print k[0:3],pattern
                if k[0:3]==pattern or k[0:3]==[rev[l[0]]+l[1:] for l in pattern]:
                    mygames.append(i)
                    patterns.append(k)
    return patterns,mygames

def WritePatterns(patterns,mygames,path='./outputs/',name='test'):
    header="(;GM[1]FF[4]CA[UTF-8]ST[2]SZ[19]"
    for i,game in zip(patterns,mygames):
        if len(i)==0:continue
        tmp='(;C['+game+']'
        for j in i[:-1]:
            j=j.replace(')','')
            j=j.replace('(','')
            tmp+=j+';'
        i[-1]=i[-1].replace('(','')
        tmp+=i[-1].replace(')','')   
        tmp+=')\n'
        header+=tmp
    f=open(path+name+'.sgf','w')
    f.write(header+')')
    f.close()
    
def WriteOrderedPatterns(patterns,mygames,path='./outputs/',name='test'):
    header="(;GM[1]FF[4]CA[UTF-8]ST[2]SZ[19]"
    for i,game in zip(patterns,mygames):
        if len(i)==0:continue
        tmp='(;C['+game+']'
        for j in i[:-1]:
            j=j.replace(')','')
            j=j.replace('(','')
            tmp+=j+';'
        i[-1]=i[-1].replace('(','')
        tmp+=i[-1].replace(')','')   
        tmp+=')\n'
        header+=tmp
    f=open(path+name+'.sgf','w')
    f.write(header+')')
    f.close()

f=open('./GoDat/KGS/Borggren.games')
mygames=f.read().split('\n')
games = [i for i in mygames if i.find('.sgf')>0 and i.find('/Bogoliubon-7.sgf')<0 and i.find('/Bogoliubon-6.sgf')<0 and i.find('/Bogoliubon-3.sgf')<0 and i.find('/Bogoliubon-5.sgf')<0 and i.find('/Bogoliubon-4.sgf')<0 and i.find('/Bogoliubon.sgf')<0 and i.find('/Bogoliubon-2.sgf')<0]
myana = [i for i in mygames if i.find('/Bogoliubon.sgf')>0]
print len(games),len(mygames)
games.reverse()

5086 7229


In [11]:
f=open('./inputs/logicg8.games')
mygames=f.read().split('\n')
games = [i for i in mygames if i.find('.sgf')>0 and i.find('/Logicg8.sgf')<0]
games.reverse()
print len(games),len(mygames)

2366 3061


In [2]:
f = open('/Users/nborggren/Codes/ipy/go/go4go/pro.list')
mygames=f.read().split('\n')
games = [i for i in mygames if i.find('.sgf')>0]
games.reverse()
print len(games),len(mygames)

48689 48693


In [250]:
corners=[]
for i in games:
    CleanSGF(i,path='/Users/nborggren/Codes/ipy/go/go4go/')
    #patterns,pgames=PatternSearch(['W[cd]', 'B[ed]', 'W[hd]'],games[:3],path='/Users/nborggren/Codes/ipy/go/go4go/')

IOError: [Errno 2] No such file or directory: '/Users/nborggren/Codes/ipy/go/go4go/2016/2/24/Bogoliubon-tf.sgf'

In [222]:
GetSyms(corners[0],src='list')

[['B[dc]', 'B[cf]', 'W[ic]', 'B[gc]', 'W[ie]', 'W[ch]', 'W[cd]', 'B[cc]', 'W[ce]', 'B[cg]', 'W[ed]', 'B[ec]', 'W[dh]', 'B[ef]', 'W[ee]', 'B[ge]', 'W[eg]', 'B[ff]', 'W[df]', 'B[dg]', 'W[de]', 'B[bh]', 'W[bi]', 'B[eh]', 'W[fg]', 'B[ci]', 'W[gf]', 'B[di]', 'W[fe]', 'B[ib]', 'W[jb]', 'B[hb]', 'B[jc]', 'B[jd]', 'B[id]', 'B[he]', 'B[ig]', 'W[bd]', 'W[gi]', 'B[hf]', 'B[gg]', 'W[fh]'], ['W[cd]', 'B[fc]', 'W[hc]', 'B[jc]', 'W[dc]', 'W[gd]', 'B[di]', 'B[jh]', 'W[ji]', 'B[jg]', 'W[cg]', 'B[df]', 'W[cf]', 'B[fd]', 'W[fe]', 'B[ge]', 'W[he]', 'B[gf]', 'W[ee]', 'B[hf]', 'W[ie]', 'B[hb]', 'W[ib]', 'B[gb]', 'W[jb]', 'B[jd]', 'B[ic]', 'B[eb]', 'W[db]', 'B[bh]', 'W[ch]', 'B[ci]'], ['W[dd]', 'B[jc]', 'B[fc]', 'B[cf]', 'W[df]', 'B[dg]', 'W[ef]', 'B[ce]', 'W[cd]', 'B[gd]', 'W[ec]', 'B[eb]', 'W[fd]', 'B[fb]', 'W[db]', 'B[gf]', 'W[gg]', 'B[ch]', 'W[eg]', 'B[hg]', 'W[di]', 'B[ci]', 'W[dj]', 'B[cj]', 'W[hh]', 'B[gh]', 'W[fg]', 'B[ig]', 'W[ih]', 'B[jh]', 'W[ji]', 'B[jg]', 'W[ij]', 'W[jb]', 'B[jd]', 'W[ic]', 'B[i

KeyError: 'B[gc]'

In [149]:
sgf=ReadGame(games[0],path='./GoDat/KGS_logicg8/')

In [161]:
for i in games:CleanSGF(i,path='./GoDat/KGS_logicg8/')

In [31]:
for i in games:StripComments(i)

In [34]:
games[0]

'2016/2/24/Bogoliubon-tf.sgf'

In [139]:
for i in games:CleanSGF(i)

In [140]:
sgf=ReadGame(games[0].replace('.sgf','_clean.sgf'))

In [217]:
corners=GetCorners(ReadGame(games[0].replace('.sgf','_clean.sgf')))
WriteCorners(corners[0],name='0_0')
syms=GetSyms(ReadGame(games[0].replace('.sgf','_clean.sgf')))
for i,j in enumerate(syms):WriteCorners(syms[i],name='syms_0_'+str(i))

In [90]:
letters='abcdefghijklmnopqrs'
bletters='srqponmlkjihgfedcba'
sym={}
for i,j in zip(letters,bletters):
    sym[i]=j

In [76]:
l,r=letters[0:10],letters[9:]

In [161]:
c1=[]
c2=[]
c3=[]
c4=[]
for i,j in zip(moves,locs):
    if j[0] in l and j[1] in l:
        c1.append(i)
    if j[0] in l and j[1] in r:
        tmp=i.replace(j[1],sym[j[1]])
        c2.append(tmp)
    if j[0] in r and j[1] in r:
        tmp=i.replace(j[1],sym[j[1]])
        tmp=tmp.replace(j[0],sym[j[0]])
        c3.append(tmp)
    if j[0] in r and j[1] in l:
        tmp=i.replace(j[0],sym[j[0]])
        c4.append(tmp)


In [103]:
header="(;GM[1]FF[4]CA[UTF-8]AP[CGoban:3]ST[2]\nRU[Japanese]SZ[19]KM[6.50]TM[600]OT[5x40 byo-yomi]\nPW[Bogoliubon]PB[tf]WR[1k]BR[1k]DT[2016-02-24]PRE[B+Resign]\n;"

In [142]:
def WriteCorners(corners,path='./GoDat/Corners/'):
    header="(;GM[1]FF[4]CA[UTF-8]AP[CGoban:3]ST[2]\nRU[Japanese]SZ[19]KM[6.50]TM[600]OT[5x40 byo-yomi]\nPW[Bogoliubon]PB[tf]WR[1k]BR[1k]DT[2016-02-24]PRE[B+Resign]\n;"

In [143]:
header

'(;GM[1]FF[4]CA[UTF-8]AP[CGoban:3]ST[2]\nRU[Japanese]SZ[19]KM[6.50]TM[600]OT[5x40 byo-yomi]\nPW[Bogoliubon]PB[tf]WR[1k]BR[1k]DT[2016-02-24]PRE[B+Resign]\n;'

In [112]:
f=open('test.sgf','w')
record=header
for i in c1[:-1]:record+=i+';'
record+=c1[-1].replace(')','')+')'

record+='\n'+header
for i in c2[:-1]:record+=i+';'
record+=c2[-1].replace(')','')+')'

record+='\n'+header
for i in c1[:-1]:record+=i+';'
record+=c1[-1].replace(')','')+')'

record+='\n'+header
for i in c1[:-1]:record+=i+';'
record+=c1[-1].replace(')','')+')'

f.write(record)

In [115]:
corners=[c1,c2,c3,c4]

In [119]:
joseki={}
joseki[c1[0]]={}
joseki[c1[0]][c1[1]]={}
        


In [120]:
joseki

{'W[dc]': {'B[de]': {}}}

In [121]:
fmove=set()
for i in corners:
    fmove.add(i[0])
    

In [122]:
print fmove

set(['W[dd]', 'W[dc]', 'B[dd]'])


In [172]:
corners=GetCorners(ReadGame(games[3],path='/Users/nborggren/Codes/ipy/go/go4go/'))

In [13]:
for j,i in enumerate(games):
    sgf=ReadGame(i.replace('.sgf','_clean.sgf'))
    handis=GetKey(sgf='HA')
    if handis in [str(k+1) for k in range(9)]:
        continue
    else:
        try:
            corners=GetCorners(sgf)
        except IndexError:
            continue
        for k,l in enumerate(corners):
            WriteCorners(l,name=str(j)+'_'+str(k))


In [ ]:
f=open('test.sgf','w')
record=header
for i in c1[:-1]:record+=i+';'
record+=c1[-1].replace(')','')+')'

In [150]:
WriteCorners(corners[0])

In [25]:
a=GetSyms(corners[1],src='list')

In [33]:
a[4][0:3]==['W[cd]', 'B[ed]', 'W[gc]'] or a[4][0:3]==['B[cd]', 'W[ed]', 'B[gc]']

True

In [92]:
def PatternSearch(pattern,games):
    patterns=[]
    mygames=[]
    rev = {}
    rev['B']='W'
    rev['W']='B'
    for i in games:
        try:
            corners=GetCorners(ReadGame(i.replace('.sgf','_clean.sgf')))
        except IndexError:
            continue
        for j in corners:
            a=GetSyms(j,src='list')
            for k in a:
                if k[0:3]==pattern or k[0:3]==[rev[l[0]]+l[1:] for l in pattern]:
                    mygames.append(i)
                    patterns.append(k)
    return patterns,mygames

In [125]:
patterns,mygames=PatternSearch(['W[cd]', 'B[ed]', 'W[gd]'],games)

In [164]:
patterns,mygames=PatternSearch(['W[cd]', 'B[ed]', 'W[gd]'],games,path='./GoDat/KGS_logicg8/')
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kha1h',path='./outputs/logicg8/')

patterns,mygames=PatternSearch(['W[cd]', 'B[ed]', 'W[gc]'],games,path='./GoDat/KGS_logicg8/')
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kha1l',path='./outputs/logicg8/')

patterns,mygames=PatternSearch(['W[cd]', 'B[ed]', 'W[hd]'],games,path='./GoDat/KGS_logicg8/')
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kha2h',path='./outputs/logicg8/')

patterns,mygames=PatternSearch(['W[cd]', 'B[ed]', 'W[hc]'],games,path='./GoDat/KGS_logicg8/')
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kha2l',path='./outputs/logicg8/')

patterns,mygames=PatternSearch(['W[cd]', 'B[ed]', 'W[id]'],games,path='./GoDat/KGS_logicg8/')
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kha3h',path='./outputs/logicg8/')

patterns,mygames=PatternSearch(['W[cd]', 'B[ed]', 'W[ic]'],games,path='./GoDat/KGS_logicg8/')
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kha3l',path='./outputs/logicg8/')



35 35
45 45
140 140
1 1
0 0
1 1


In [153]:
patterns,mygames=PatternSearch(['W[cd]', 'B[ec]', 'W[gd]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kla1h')

patterns,mygames=PatternSearch(['W[cd]', 'B[ec]', 'W[gc]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kla1l')

patterns,mygames=PatternSearch(['W[cd]', 'B[ec]', 'W[hd]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kla2h')

patterns,mygames=PatternSearch(['W[cd]', 'B[ec]', 'W[hc]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kla2l')

patterns,mygames=PatternSearch(['W[cd]', 'B[ec]', 'W[id]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kla3h')

patterns,mygames=PatternSearch(['W[cd]', 'B[ec]', 'W[ic]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kla3l')


29 29
140 140
79 79
57 57
8 8
28 28


In [199]:
corners=[]
for i in mygames[:3]:
    corners.append(GetCorners(ReadGame(i,path='/Users/nborggren/Codes/ipy/go/go4go/')))
    
patterns,pgames=PatternSearchCorners(corners,['W[cd]', 'B[ed]', 'W[hd]'],mygames[:3])

In [230]:
len(games)

48689

In [254]:
patterns,mygames=PatternSearch(['W[dd]', 'B[fc]', 'W[cf]','B[db]','W[gc]'],games,path='/Users/nborggren/Codes/ipy/go/go4go/')
print len(patterns),len(mygames)
n=len(patterns)/300
for i in range(n+1):
    WritePatterns(patterns[i*300:min([len(patterns),(i+1)*300])],mygames[i*300:min([len(patterns),(i+1)*300])],name='hlakeimaslideattach_'+str(i),path='./outputs/pros/')
    

760 760


In [4]:
patterns,mygames=PatternSearch(['B[ed]'],games,path='/Users/nborggren/Codes/ipy/go/go4go/')
print len(patterns),len(mygames)
n=len(patterns)/300
for i in range(n+1):
    WritePatterns(patterns[i*300:min([len(patterns),(i+1)*300])],mygames[i*300:min([len(patterns),(i+1)*300])],name='5-4_'+str(i),path='./outputs/pros/')
    

678 678


In [9]:
sorted(list(smoves),key=lambda x: cmoves.count(x))

['B[ge]',
 'B[eg]',
 'B[hc]',
 'W[jb]',
 'W[ch]',
 'B[hd]',
 'W[ij]',
 'B[ii]',
 'B[cg]',
 'B[ci]',
 'W[hj]',
 'B[je]',
 'W[id]',
 'W[jj]',
 'B[dh]',
 'W[gj]',
 'W[hc]',
 'W[ei]',
 'B[jf]',
 'B[ih]',
 'W[ji]',
 'B[gj]',
 'B[gi]',
 'W[jg]',
 'W[gi]',
 'B[id]',
 'B[jj]',
 'B[hj]',
 'B[ej]',
 'W[cg]',
 'B[jh]',
 'W[je]',
 'B[ic]',
 'B[ji]',
 'W[di]',
 'W[jd]',
 'W[dj]',
 'W[ci]',
 'B[di]',
 'B[dj]',
 'B[jd]',
 'W[cj]',
 'W[ce]',
 'B[jc]',
 'B[gc]',
 'B[cj]',
 'W[cc]',
 'W[jc]',
 'W[gc]',
 'B[ce]',
 'B[cc]',
 'W[cd]',
 'B[cd]']

In [5]:
cmoves=[i[1] for i in patterns]
smoves=set(cmoves)
for i in smoves:
    print i,cmoves.count(i)


B[gc] 12
W[gi] 2
W[jd] 5
B[jh] 3
B[dj] 7
B[ge] 1
B[eg] 1
B[hc] 1
B[id] 2
W[jb] 1
B[jj] 2
W[ch] 1
B[hd] 1
W[cj] 9
B[hj] 2
W[ij] 1
B[ii] 1
B[jc] 11
B[cc] 50
W[dj] 5
B[cg] 1
B[ci] 1
W[hj] 1
B[je] 1
W[id] 1
B[di] 6
B[ce] 30
W[cd] 152
W[jj] 1
W[cc] 18
W[jc] 19
B[ji] 4
W[je] 3
B[dh] 1
W[gj] 1
W[hc] 1
B[ic] 3
W[ei] 1
W[ci] 5
W[di] 4
W[gc] 28
B[jf] 1
B[ej] 2
B[ih] 1
B[cd] 237
W[ce] 10
W[ji] 1
B[gj] 1
B[gi] 1
B[cj] 14
B[jd] 7
W[cg] 2
W[jg] 1


In [257]:
patterns,mygames=PatternSearch(['B[dd]', 'B[fc]','B[ce]'],games,path='/Users/nborggren/Codes/ipy/go/go4go/')
print len(patterns),len(mygames)
n=len(patterns)/300
for i in range(n+1):
    WritePatterns(patterns[i*300:min([len(patterns),(i+1)*300])],mygames[i*300:min([len(patterns),(i+1)*300])],name='hoshienclosure_'+str(i),path='./outputs/pros/')
    

375 375


In [244]:
n=len(patterns)/300
for i in range(n+1):
    WritePatterns(patterns[i*300:min([len(patterns),(i+1)*300])],mygames[i*300:min([len(patterns),(i+1)*300])],name='kla1l_'+str(i),path='./outputs/pros/')
    

In [165]:
patterns,mygames=PatternSearch(['W[cd]', 'B[ec]', 'W[gd]'],games,path='./GoDat/KGS_logicg8/')
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kla1h',path='./outputs/logicg8/')

patterns,mygames=PatternSearch(['W[cd]', 'B[ec]', 'W[gc]'],games,path='./GoDat/KGS_logicg8/')
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kla1l',path='./outputs/logicg8/')

patterns,mygames=PatternSearch(['W[cd]', 'B[ec]', 'W[hd]'],games,path='./GoDat/KGS_logicg8/')
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kla2h',path='./outputs/logicg8/')

patterns,mygames=PatternSearch(['W[cd]', 'B[ec]', 'W[hc]'],games,path='./GoDat/KGS_logicg8/')
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kla2l',path='./outputs/logicg8/')

patterns,mygames=PatternSearch(['W[cd]', 'B[ec]', 'W[id]'],games,path='./GoDat/KGS_logicg8/')
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kla3h',path='./outputs/logicg8/')

patterns,mygames=PatternSearch(['W[cd]', 'B[ec]', 'W[ic]'],games,path='./GoDat/KGS_logicg8/')
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kla3l',path='./outputs/logicg8/')


5 5
80 80
14 14
45 45
4 4
142 142


In [132]:
patterns,mygames=PatternSearch(['W[dd]', 'B[fd]', 'W[hd]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hha1h')

patterns,mygames=PatternSearch(['W[dd]', 'B[fd]', 'W[hc]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hha1l')

patterns,mygames=PatternSearch(['W[dd]', 'B[fd]', 'W[id]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hha2h')

patterns,mygames=PatternSearch(['W[dd]', 'B[fd]', 'W[ic]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hha2l')

patterns,mygames=PatternSearch(['W[dd]', 'B[fc]', 'W[hd]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hla1h')

patterns,mygames=PatternSearch(['W[dd]', 'B[fc]', 'W[hc]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hla1l')

patterns,mygames=PatternSearch(['W[dd]', 'B[fc]', 'W[id]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hla2h')

patterns,mygames=PatternSearch(['W[dd]', 'B[fc]', 'W[ic]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hla2l')



1 1
8 8
2 2
0 0
35 35
444 444
183 183
79 79


In [166]:
patterns,mygames=PatternSearch(['W[dd]', 'B[fd]', 'W[hd]'],games,path='./GoDat/KGS_logicg8/')
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hha1h',path='./outputs/logicg8/')

patterns,mygames=PatternSearch(['W[dd]', 'B[fd]', 'W[hc]'],games,path='./GoDat/KGS_logicg8/')
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hha1l',path='./outputs/logicg8/')

patterns,mygames=PatternSearch(['W[dd]', 'B[fd]', 'W[id]'],games,path='./GoDat/KGS_logicg8/')
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hha2h',path='./outputs/logicg8/')

patterns,mygames=PatternSearch(['W[dd]', 'B[fd]', 'W[ic]'],games,path='./GoDat/KGS_logicg8/')
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hha2l',path='./outputs/logicg8/')

patterns,mygames=PatternSearch(['W[dd]', 'B[fc]', 'W[hd]'],games,path='./GoDat/KGS_logicg8/')
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hla1h',path='./outputs/logicg8/')

patterns,mygames=PatternSearch(['W[dd]', 'B[fc]', 'W[hc]'],games,path='./GoDat/KGS_logicg8/')
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hla1l',path='./outputs/logicg8/')

patterns,mygames=PatternSearch(['W[dd]', 'B[fc]', 'W[id]'],games,path='./GoDat/KGS_logicg8/')
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hla2h',path='./outputs/logicg8/')

patterns,mygames=PatternSearch(['W[dd]', 'B[fc]', 'W[ic]'],games,path='./GoDat/KGS_logicg8/')
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hla2l',path='./outputs/logicg8/')



4 4
2 2
0 0
0 0
14 14
350 350
43 43
29 29


In [124]:
len(patterns)
WritePatterns(patterns,mygames,name='onespacehigh')

In [41]:
patterns=[]
mygames=[]
for i in games:
    try:
        corners=GetCorners(ReadGame(i.replace('.sgf','_clean.sgf')))
    except IndexError:
        continue
    for j in corners:
        a=GetSyms(j,src='list')
        for k in a:
            if k[0:3]==['W[cd]', 'B[ed]', 'W[gc]'] or k[0:3]==['B[cd]', 'W[ed]', 'B[gc]']:
                print i
                mygames.append(i)
                patterns.append(k)

2016/2/22/takyan-Bogoliubon.sgf
2016/2/22/ibara-Bogoliubon.sgf
2016/1/15/Bogoliubon-AyaBot05.sgf
2016/1/11/AyaBot6-Bogoliubon.sgf
2016/1/1/AyaMC2-Bogoliubon.sgf
2015/7/21/Bogoliubon-AyaBot02-2.sgf
2015/12/20/darkfores3-Bogoliubon.sgf
2015/12/19/MCark2-Bogoliubon.sgf
2015/12/17/Bogoliubon-AyaBot05.sgf
2014/8/17/mobi-Bogoliubon.sgf
2014/5/8/Bogoliubon-GoraUralsk.sgf
2014/5/25/Nilpferd-Bogoliubon.sgf
2014/4/22/matta-Bogoliubon.sgf
2014/3/8/Archaic-Bogoliubon.sgf
2014/2/2/mausakim-Bogoliubon.sgf
2013/9/21/jco-Bogoliubon.sgf
2013/9/15/boa-Bogoliubon.sgf
2013/9/13/Ubiquity-Bogoliubon.sgf
2013/9/13/Bogoliubon-fcwr.sgf
2013/9/12/AyaBot04-Bogoliubon.sgf
2013/8/23/clee-Bogoliubon.sgf
2013/8/10/Request-Bogoliubon.sgf
2013/8/10/Bogoliubon-6.sgf
2013/7/6/Bogoliubon-facedi.sgf
2013/6/15/asa14-Bogoliubon.sgf
2013/3/5/arnoblitz-Bogoliubon.sgf
2013/12/6/Bogoliubon-peter2.sgf
2013/12/25/Bogoliubon-Nigeteiru.sgf
2013/12/15/Prezli-Bogoliubon.sgf
2013/11/6/LDD99-Bogoliubon.sgf
2013/11/3/paulfac-Bogoliubon.

In [38]:
for i in patterns:print i

['W[cd]', 'B[ed]', 'W[gc]', 'B[cc]', 'W[dc]', 'B[dd]', 'W[ec]', 'B[fc]', 'W[fd]', 'B[fb]', 'W[cb]', 'B[bc]', 'W[eb]', 'B[gb]', 'W[ce]', 'B[ea]', 'W[db]', 'B[df]', 'W[bb]', 'B[hc]', 'W[de]', 'B[ee]', 'W[ef]', 'B[fe]', 'W[ff]', 'B[ge]', 'W[dg]', 'B[gf]', 'B[ci]', 'B[bh]', 'B[cf]', 'W[bd]', 'B[be]', 'W[ac]', 'B[cg]']
['W[cd]', 'B[ed]', 'W[gc]', 'B[cc]', 'W[cg]', 'B[dd]', 'W[bd]', 'B[eb]', 'W[jc]', 'B[bc]', 'B[ci]', 'B[eh]', 'W[ef]', 'W[cj]', 'B[ch]', 'W[dj]', 'B[ej]', 'W[eg]', 'B[id]', 'W[ic]', 'B[jd]', 'B[fe]', 'W[fh]', 'B[ei]', 'W[gf]', 'B[hd]', 'W[gd]', 'B[ge]', 'W[hh]', 'W[hf]', 'B[je]', 'W[gj]', 'W[bg]', 'B[hj]', 'B[ij]', 'B[gi]', 'W[fi]', 'B[fg]', 'W[gh]', 'B[ff]', 'W[bi]', 'B[bh]', 'W[ah]', 'B[bf]', 'W[cf]', 'B[ce]', 'W[be]', 'B[af]', 'W[de])']


In [127]:

def WritePatterns(patterns,mygames,path='./outputs/',name='test'):
    header="(;GM[1]FF[4]CA[UTF-8]ST[2]SZ[19]"
    for i,game in zip(patterns,mygames):
        if len(i)==0:continue
        tmp='(;C['+game+']'
        for j in i[:-1]:
            j=j.replace(')','')
            j=j.replace('(','')
            tmp+=j+';'
        i[-1]=i[-1].replace('(','')
        tmp+=i[-1].replace(')','')   
        tmp+=')\n'
        header+=tmp
    f=open(path+name+'.sgf','w')
    f.write(header+')')
    f.close()

In [84]:
len(patterns)

234

In [128]:
WritePatterns(patterns,mygames)

In [168]:
print patterns[3]

['B[dd]', 'W[fc]', 'B[ic]', 'W[cc]', 'B[dc]', 'W[cd]', 'B[de]', 'W[db]', 'B[eb]', 'W[cb]', 'B[ec]', 'W[cf]', 'B[eg]', 'W[cg]', 'B[ej]', 'B[hi]', 'W[gj]', 'B[gi]', 'W[fj]', 'B[fi]', 'W[di]', 'B[ei]', 'W[dj]', 'W[jj]', 'W[ij]', 'B[jg]', 'W[jb]', 'B[ib]', 'W[jc]', 'W[id]', 'B[je]', 'B[jf]', 'W[jd]', 'B[gb]', 'W[hc]', 'B[hb]', 'W[fe]', 'B[ef]', 'W[gf]', 'B[gc]', 'W[gd]', 'B[hd]', 'W[he]', 'B[ie]', 'W[hg]', 'B[hc]', 'W[fb]', 'B[fa]', 'W[fd]', 'B[ga]', 'W[ia]', 'B[ih])']
